In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats



In [7]:
skincare_data = pd.read_csv("00. InfoProduct.csv")
#skincare_data = skincare_data['Reviewer'].drop()

skincare_data=skincare_data.drop(['Reviewer'], axis=1)
skincare_data.head()
#lowercase column titles
#categorize columns by Category
#categorize by Merk
#rename merk --> brand
#Price: Rp --> USD
#groupby brands
#average ratings per Category/Merk
    #boxplots


,Category,Merk,Product,Price,OverallRating
0,Night Cream,Hada Labo,Perfect 3D Gel,Rp. 91.000,3.8
1,Face Mist,PIXY,Aqua Beauty Protecting Mist,Rp. 27.000,4.2
2,Night Cream,Wardah,Lightening Night Cream,Rp. 0,3.5
3,Face Mist,Avene,Thermal Spring Water,Rp. 199.000,4.4
4,Night Cream,Wardah,White Secret Night Cream,Rp. 98.000,3.6
